In [1]:
from rule_based_segmentation import mask_non_skin
from ita_extraction import ITA, empirical_classification, kinyananjui_classification
import os
import pandas as pd
import imageio as im
import time

In [2]:
def func(file_name,real_class):
    image = im.imread(file_name)
    direct_ita = ITA(image)
    skin,__ = mask_non_skin(image)
    segmented_ita = ITA(skin)
    
    #kinyananjui_class_direct_ita = kinyananjui_classification(direct_ita)
    #empirical_class_direct_ita = empirical_classification(direct_ita)
    kinyananjui_class_segmented_ita = kinyananjui_classification(segmented_ita)
    empirical_class_segmented_ita = empirical_classification(segmented_ita)
    
    #direct_kinyananjui_hit = 1 if kinyananjui_class_direct_ita == real_class else 0
    #direct_empirical_hit = 1 if empirical_class_direct_ita == real_class else 0
    kinyananjui_hit = 1 if kinyananjui_class_segmented_ita == real_class else 0
    empirical_hit = 1 if empirical_class_segmented_ita == real_class else 0
    
    
    #direct_kinyananjui_relaxed_hit = 1 if kinyananjui_class_direct_ita >= real_class-1 and kinyananjui_class_direct_ita <= real_class+1 else 0
    #direct_empirical_relaxed_hit = 1 if empirical_class_direct_ita >= real_class-1 and empirical_class_direct_ita <= real_class+1 else 0
    kinyananjui_relaxed_hit = 1 if kinyananjui_class_segmented_ita >= real_class-1 and kinyananjui_class_segmented_ita <= real_class+1 else 0
    empirical_relaxed_hit = 1 if empirical_class_segmented_ita >= real_class-1 and empirical_class_segmented_ita <= real_class+1 else 0
    
    return segmented_ita,direct_ita,kinyananjui_class_segmented_ita,empirical_class_segmented_ita,kinyananjui_hit,empirical_hit,kinyananjui_relaxed_hit,empirical_relaxed_hit,real_class

In [3]:
df_test = pd.read_csv('../../datasets/fitzpatrick17k/fitzpatrick17k_train.csv')
print("Número de instâncias com -1 no label:"+str(len(df_test[df_test['fitzpatrick']== -1])))
print("Dropando estas instâncias...")
df_test = df_test.drop(df_test[df_test['fitzpatrick']== -1].index,axis=0)
df_test

Número de instâncias com -1 no label:423
Dropando estas instâncias...


,md5hash,fitzpatrick,label,nine_partition_label,three_partition_label,url,url_alphanum
0,36c7e8901daa48c89e2521219e6fc4b9,4,nematode infection,inflammatory,non-neoplastic,http://atlasdermatologico.com.br/img?imageId=3146,httpwwwatlasdermatologicocombrimgimageId3146.jpg
1,d3764c4118b4c2e9258fd97edb05e0f9,2,lyme disease,inflammatory,non-neoplastic,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpiceerytr...
2,8c6058baf8fe1bcc2e23e9ba6ea3a545,5,xeroderma pigmentosum,genodermatoses,non-neoplastic,http://atlasdermatologico.com.br/img?imageId=8149,httpwwwatlasdermatologicocombrimgimageId8149.jpg
3,add356472252f6e2b46d193dd7cb3620,4,granuloma annulare,inflammatory,non-neoplastic,http://atlasdermatologico.com.br/img?imageId=163,httpwwwatlasdermatologicocombrimgimageId163.jpg
4,596f4e8d75aca205fdd9f1b91dc1bbc8,4,fordyce spots,benign dermal,benign,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicffordy...
...,...,...,...,...,...,...,...
12388,795ac369716070ca60d916ab51e50d0b,1,lupus erythematosus,inflammatory,non-neoplastic,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicLlupus...
12389,1c33b61430df6797f751f7f96ef23a19,4,epidermolysis bullosa,genodermatoses,non-neoplastic,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpiceepide...
12390,7912ab0cf1c1a04708f78e68ebee8d34,2,basal cell carcinoma,malignant epidermal,malignant,http://atlasdermatologico.com.br/img?imageId=540,httpwwwatlasdermatologicocombrimgimageId540.jpg
12391,937c3214f849de5e10b823e04adfa723,3,lichen planus,inflammatory,non-neoplastic,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicLliche...


In [4]:
image_dir = '../../datasets/fitzpatrick17k/images'

In [5]:
df_result = pd.DataFrame()
df_result

""


In [6]:
start = time.time()
result = df_test.apply(lambda row: func(os.path.join(image_dir,row['md5hash']+'.jpg'),row['fitzpatrick']),axis=1)
stop = time.time()
stop-start

/opt/conda/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/store/vrusso/skin-tone-research/baseline_model/ita_extraction.py:39: RuntimeWarning: Mean of empty slice
  std, mean = np.nanstd(L), np.nanmean(L)
/store/vrusso/skin-tone-research/baseline_model/ita_extraction.py:44: RuntimeWarning: Mean of empty slice
  std, mean = np.nanstd(B), np.nanmean(B)
/store/vrusso/skin-tone-research/baseline_model/ita_extraction.py:52: RuntimeWarning: Mean of empty slice
  ITA = math.atan2(np.nanmean(L) - 50, np.nanmean(B)) * (180 / np.pi)


3503.8454394340515

In [7]:
df_aux = pd.DataFrame(list(result))
df_aux

,0,1,2,3,4,5,6,7,8
0,13.719044,10.534109,5,3,0,0,1,1,4
1,45.218413,47.643004,2,1,1,0,1,1,2
2,19.028076,30.279799,4,3,0,0,1,0,5
3,6.021998,5.961789,6,4,0,1,0,1,4
4,23.974702,23.300510,4,2,1,0,1,0,4
...,...,...,...,...,...,...,...,...,...
11965,38.697197,50.690900,3,2,0,0,0,1,1
11966,-22.430278,-8.165186,6,5,0,0,0,1,4
11967,35.054945,34.866069,3,2,0,1,1,1,2
11968,2.360832,1.990932,6,4,0,0,0,1,3


In [8]:
df_result['segmented_ita'] = df_aux[0]
df_result['direct_ita'] = df_aux[1]
df_result['kinyananjui_class'] = df_aux[2]
df_result['empirical_class'] = df_aux[3]
df_result['kinyananjui_hit'] = df_aux[4]
df_result['empirical_hit'] = df_aux[5]
df_result['kinyananjui_relaxed_hit'] = df_aux[6]
df_result['empirical_relaxed_hit'] = df_aux[7]
df_result['real_class'] = df_aux[8]



df_result

,segmented_ita,direct_ita,kinyananjui_class,empirical_class,kinyananjui_hit,empirical_hit,kinyananjui_relaxed_hit,empirical_relaxed_hit,real_class
0,13.719044,10.534109,5,3,0,0,1,1,4
1,45.218413,47.643004,2,1,1,0,1,1,2
2,19.028076,30.279799,4,3,0,0,1,0,5
3,6.021998,5.961789,6,4,0,1,0,1,4
4,23.974702,23.300510,4,2,1,0,1,0,4
...,...,...,...,...,...,...,...,...,...
11965,38.697197,50.690900,3,2,0,0,0,1,1
11966,-22.430278,-8.165186,6,5,0,0,0,1,4
11967,35.054945,34.866069,3,2,0,1,1,1,2
11968,2.360832,1.990932,6,4,0,0,0,1,3


In [9]:
df_result.to_csv('baseline_results_train_split.csv',index=False)